# Sentimentanalyse von Plenarprotokollen mit Python

Dieses Dokument dient zum Erkunden der Datensätze, die mithilfe der in diesem Ordner verfügbaren Skriptdateien ``process_pp.py`` und ``dictionary_analysis.py`` erstellt wurden. Der größte Teil der Rechenarbeit wurde also mithilfe dieser Skripte bereits durchgeführt. Das Laden des aufbereiteten Datensatzes kann trotzdem einige Minuten dauern.

Zunächst sollen alle im Folgenden benötigte Python-Pakete geladen werden. Mit dem Code in der nächsten Zelle kann überprüft werden, ob diese Pakete installiert sind. Zusätzlich soll ein deutsches Sprachmodell ``de_core_news_sm`` für spaCy heruntergeladen werden.

In [ ]:
!pip install spacy pandas
!python -m spacy download de_core_news_sm

Als nächstes werden die installierten Pakete für das aktuelle Skript importiert und das Sprachmodell verfügbar gemacht.

In [ ]:
import spacy
from spacy.tokens import DocBin
nlp = spacy.load("de_core_news_sm")
import pandas as pd
from glob import glob

Anschließend werden die für die Analyse nötigen Daten eingelesen. Dieser Prozess kann einige Zeit in Anspruch nehmen.

In [ ]:
dateien = glob("plenarprotokolle/testing_prep/*.xml.spacy.sentiment")
spacy_db = {}
for protokoll in dateien:
    protokoll_daten = DocBin(store_user_data=True).from_bytes(open(protokoll, "rb").read())
    protokoll_daten = list(protokoll_daten.get_docs(nlp.vocab))
    spacy_db[protokoll] = protokoll_daten

Nun sind alle für die Analyse nötigen Ressourcen verfügbar. Zum Einen kann jetzt natürlich mit diesen experimentiert werden, zum Anderen bietet der Rest dieses Dokuments eine Führung durch beispielhafte Analysevorgänge. Zum Beispiel können grundlegende Statistiken zum Datensatz zusammengestellt werden:

In [ ]:
print("Der Datensatz enthält", len(spacy_db), "Protokolle.")

anzahl_reden = 0
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        anzahl_reden += 1
print("Diese Protokolle enthalten insgesamt", anzahl_reden, "Reden.")

anzahl_token = 0
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        anzahl_token += len(rede)
print("Diese Reden enthalten insgesamt", anzahl_token, "Token.")

In [ ]:
redner_parteien = {}
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        partei = rede.user_data["meta"]["redner_partei"]
        if not partei in redner_parteien.keys():
            redner_parteien[partei] = [rede.user_data["meta"]["redner_name"]]
        else:
            redner_parteien[partei].append(rede.user_data["meta"]["redner_name"])

for partei, namen in redner_parteien.items():
    print(partei, ":", len(namen))

Zunächst sollen alle im Folgenden benötigte Python-Pakete geladen werden. Mit dem Code in der nächsten Zelle kann überprüft werden, ob diese Pakete installiert sind. Zusätzlich soll ein deutsches Sprachmodell ``de_core_news_sm`` für spaCy heruntergeladen werden.

In [66]:
!pip install spacy pandas
!python -m spacy download de_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


Als nächstes werden die installierten Pakete für das aktuelle Skript importiert und das Sprachmodell verfügbar gemacht.

In [67]:
import spacy
from spacy.tokens import DocBin
nlp = spacy.load("de_core_news_sm")
import pandas as pd
from glob import glob

Anschließend werden die für die Analyse nötigen Daten eingelesen. Dieser Prozess kann einige Zeit in Anspruch nehmen.

In [74]:
dateien = glob("plenarprotokolle/testing_prep/*.xml.spacy.sentiment")
spacy_db = {}
for protokoll in dateien:
    protokoll_daten = DocBin(store_user_data=True).from_bytes(open(protokoll, "rb").read())
    protokoll_daten = list(protokoll_daten.get_docs(nlp.vocab))
    spacy_db[protokoll] = protokoll_daten

Nun sind alle für die Analyse nötigen Ressourcen verfügbar. Zum Einen kann jetzt natürlich mit diesen experimentiert werden, zum Anderen bietet der Rest dieses Dokuments eine Führung durch beispielhafte Analysevorgänge. Zum Beispiel können grundlegende Statistiken zum Datensatz zusammengestellt werden:

In [79]:
print("Der Datensatz enthält", len(spacy_db), "Protokolle.")

anzahl_reden = 0
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        anzahl_reden += 1
print("Diese Protokolle enthalten insgesamt", anzahl_reden, "Reden.")

anzahl_token = 0
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        anzahl_token += len(rede)
print("Diese Reden enthalten insgesamt", anzahl_token, "Token.")

Der Datensatz enthält 10 Protokolle.
Diese Protokolle enthalten insgesamt 659 Reden.
Diese Reden enthalten insgesamt 428125 Token.


In [86]:
redner_parteien = {}
for datei, protokoll in spacy_db.items():
    for rede in protokoll:
        partei = rede.user_data["meta"]["redner_partei"]
        if not partei in redner_parteien.keys():
            redner_parteien[partei] = [rede.user_data["meta"]["redner_name"]]
        else:
            redner_parteien[partei].append(rede.user_data["meta"]["redner_name"])

for partei, namen in redner_parteien.items():
    print(partei, ":", len(namen))

FDP : 81
SPD : 148
AfD : 84
DIE LINKE. : 73
CDU : 138
BÜNDNIS 90/DIE GRÜNEN : 78
CSU : 50
Unbekannt : 2
Plos : 5
